In [1]:
import torch
d = 10240
dimen = (d,)

In [2]:
concepts = ['country', 'currency', 'cap']
countries = ['usa', 'chn']
currency = ['dol', 'rmb']
cap = ['wsd', 'bei']

hv_feature = {f: 2 * torch.randint(low=0, high=2, size=dimen) - 1 for f in concepts}
hv_country = {c: 2 * torch.randint(0, 2, dimen) - 1 for c in countries}
hv_currency = {c: 2 * torch.randint(0, 2, dimen) - 1 for c in currency}
hv_cap =  {c: 2 * torch.randint(0, 2, dimen) - 1 for c in cap}

# usa = torch.bernoulli(torch.full(dimen, 0.5))

print(hv_country['usa'])
print(hv_feature['country'])

tensor([-1, -1,  1,  ...,  1, -1, -1])
tensor([-1,  1,  1,  ...,  1,  1, -1])


In [3]:
def bind(hv1, hv2):
    """
    Function
    ===
    xor elementwise
    """    
    return hv1 * hv2

def bundle(hv1, hv2):
    """
    Function
    ===
    if the sum >= 1, set to 1
    else set to 0
    """
    #return hv1 + hv2
    return ((hv1 + hv2) > 0).int() * 2 - 1

usa_HosVec = bundle(
                bind(hv_feature['country'], hv_country['usa']),
                bind(hv_feature['currency'], hv_currency['dol']))#,
               # bind(hv_feature['cap'], hv_cap['wsd']))

chn_HosVec = bundle(
                bind(hv_feature['country'], hv_country['chn']),
                bind(hv_feature['currency'], hv_currency['rmb']))#,
               # bind(hv_feature['cap'], hv_cap['bei']))


bundle 直接量化为 {-1， 1} 后，相似度难以分辨

In [4]:
print(bind(hv_feature['country'], hv_country['usa']))
print(bind(hv_feature['currency'], hv_currency['dol']))

tensor([ 1, -1,  1,  ...,  1, -1,  1])
tensor([-1, -1,  1,  ..., -1, -1,  1])


In [5]:
usa_HosVec, (usa_HosVec > 1).int()

(tensor([-1, -1,  1,  ..., -1, -1,  1], dtype=torch.int32),
 tensor([0, 0, 0,  ..., 0, 0, 0], dtype=torch.int32))

In [6]:
usa2chn = bind(usa_HosVec, chn_HosVec)
usa2chn

tensor([ 1,  1,  1,  ..., -1,  1,  1], dtype=torch.int32)

In [7]:
query = bind(hv_currency['dol'], usa2chn)
query

tensor([ 1,  1,  1,  ...,  1,  1, -1])

# cos/Humming 相似度

In [ ]:
import torch.nn.functional as F

def simi_Hum(hv1, hv2):
    """
    p = 1 Humming distance
    """
    ham = hv1 ^ hv2
    return torch.sum(ham != 0).item()

def simi_cos(hv1, hv2):
    
    return F.cosine_similarity(hv1.float(), hv2.float(), dim=0).item()

print(simi_Hum(usa_HosVec, chn_HosVec))
print(simi_Hum(query, hv_currency['rmb']))
print(simi_Hum(query, hv_currency['dol']))
print(simi_Hum(query, hv_country['usa']))
print(simi_Hum(query, hv_country['chn']))
print(simi_Hum(query, hv_feature['country']))
print(simi_Hum(query, hv_feature['currency']))

print(simi_cos(usa_HosVec, chn_HosVec))
print(simi_cos(query, hv_currency['rmb']))
print(simi_cos(query, hv_currency['dol']))
print(simi_cos(query, hv_country['usa']))
print(simi_cos(query, hv_country['chn']))
print(simi_cos(query, hv_feature['country']))
print(simi_cos(query, hv_feature['currency']))


3841
3827
3841
5129
5165
5059
6359
0.24980470538139343
0.2525390684604645
0.24980470538139343
-0.0017578131519258022
-0.008789063431322575
0.01191406324505806
-0.24199220538139343


不进行量化，计算出的 query Ham 距离无法区分，只有 cos 存在差异。

两变量的 cos 近似度 最高为 0.5，
而三变量的 cos 近似度最高 0.3

In [9]:
import torch

# 假设你有一个torch.vector
vector = torch.tensor([1.0, 2.0, 3.0, 4.0])

# 计算L1范数（曼哈顿距离）
l1_norm = torch.norm(vector, p=1)
print(f"L1 norm: {l1_norm}")

# 计算L2范数（欧几里得距离）
l2_norm = torch.norm(vector, p=2)
print(f"L2 norm: {l2_norm}")

L1 norm: 10.0
L2 norm: 5.4772257804870605


In [10]:
import torch
import torch.nn.functional as F

# 假设我们有两个向量
vector1 = torch.tensor([1.0, 2.0, 3.0])
vector2 = torch.tensor([-1.0, 2.0, -1.0])

# 计算两个向量的余弦相似度
cosine_sim = F.cosine_similarity(vector1, vector2, dim=0)

print(f"Cosine similarity: {cosine_sim.item()}")

Cosine similarity: 2.9802322387695312e-08


In [11]:
import torch

# 假设我们有两个向量
vector1 = torch.tensor([1, 0, 1, 1, 0], dtype=torch.int)
vector2 = torch.tensor([1, 1, 0, 0, 1], dtype=torch.int)

# 计算两个向量的按位异或
xor_result = vector1 ^ vector2

# 计算汉明距离，即异或结果中非零元素的数量
hamming_distance = torch.sum(xor_result != 0).item()

print(f"Hamming distance: {hamming_distance}")

Hamming distance: 4


In [7]:
from tqdm import trange
from random import random,randint
import time
 
with trange(10) as t:
  for i in t:
    #设置进度条左边显示的信息
    t.set_description("GEN %i"%i)
    #设置进度条右边显示的信息
    t.set_postfix(loss=random(),gen=randint(1,999),str="h",lst=[1,2])
    time.sleep(0.1)


GEN 9: 100%|██████████| 10/10 [00:01<00:00,  9.62it/s, gen=58, loss=0.0886, lst=[1, 2], str=h]


In [ ]:
import torch

am = torch.tensor(
    [[1, 0, 1, 1, 0],
    [0, 1, 1, 0, 0],
    [1, 0, 0, 1, 0],
    [1, 1, 1, 1, 0],
    [0, 1, 0, 0, 0],]
)

cls_simi = [len(torch.nonzero(am[i])) for i in range(am.size(0))]
cls_simi, min(cls_simi), cls_simi.index(min(cls_simi)),

([3, 2, 2, 4, 1], 1, 4)